# imports

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import re
import json
import openai
import os
import googlemaps
from pandas.api.types import is_number

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None)


# utils

In [2]:
with open('keywords.json', 'r', encoding='utf-8') as f:
    KEYWORDS = json.load(f)
    print(len(KEYWORDS))

def clean_field(val):
    try:
        if pd.isna(val) or val == '':
            return ''
        return str(val)
    except Exception as e:
        print(f"[ERROR] clean_field failed for value: {val}, error: {e}")
        return ''

def clean_city(val):
    try:
        if pd.isna(val) or val == '':
            return pd.NA
        val = str(val).lower()
        val = re.sub(r'[^a-z0-9\s]', ' ', val)
        val = re.sub(r'\s+', ' ', val)
        return val.strip()
    except Exception as e:
        print(f"[ERROR] clean_city failed for value: {val}, error: {e}")
        return pd.NA

def clean_pincode(val):
    try:
        if pd.isna(val):
            return pd.NA
        if is_number(val) == True:
            val = int(val)  # check if it's a valid number
        val = str(val)
        if len(val) != 6:
            return pd.NA
        return val
    except Exception as e:
        print(f"[ERROR] clean_pincode failed for value: {val}, error: {e}")
        return pd.NA

def clean_address(address):
    try:
        if pd.isna(address):
            return pd.NA
        address = str(address).lower()
        address = re.sub(r'[^a-z0-9\s]', ' ', address)
        address = re.sub(r'\b\d{6,}\b', '', address)
        address = re.sub(r'address line \d', '', address)
        address = re.sub(r'nullnull|nan', '', address)
        address = re.sub(r'null|nan', '', address)
        address = re.sub(r'\s+', ' ', address).strip()
        if address == '':
            return pd.NA
        return address
    except Exception as e:
        print(f"[ERROR] clean_address failed for value: {address}, error: {e}")
        return pd.NA

def clean_and_validate_address(address):
    try:
        main_address = address
        address = clean_address(address)
        if pd.isna(address):
            return False
        if len(address.split()) <= 1:
            return False
        if len(address) < 15:
            for kw in KEYWORDS:
                if kw in address:
                    return True
            if isinstance(main_address, str) and main_address.isupper() and main_address.isalpha():
                return True
            else:
                return False
        if len(address.split()) <= 6 and not re.search(r'\d', address):
            for kw in KEYWORDS:
                if kw in address:
                    return True
            return False
        return True
    except Exception as e:
        print(f"[ERROR] clean_and_validate_address failed for: {address}, error: {e}")
        return False


def clean_V_cp(df):
    df['v_series'] = df['Delivered_By_Hub_Code'].astype(str).str[1:2].eq('V').astype(int)
    return df

def data_preprocess(data):
    try:
        print("[INFO] Starting preprocessing...")

        data['address'] = data['foc_receiver_address']

        # Clean other fields
        data['foc_receiver_city'] = data['foc_receiver_city'].apply(clean_city)
        data['foc_receiver_pincode'] = data['foc_receiver_pincode'].apply(clean_pincode)
        data['address'] = data['address'].apply(clean_address)
        data['address_len'] = data['address'].str.len()
    

        cps = pd.read_csv('data/input/uq_cp_codes.csv')
        data=pd.merge(data,cps,left_on='Delivered_By_Hub_Code',right_on='0',how='left')
        #data = data[data['Delivered_By_Hub_Type']=='FR']

        # Drop NA
        before_drop = len(data)
        data.dropna(subset=['address', 'foc_receiver_pincode', 'foc_receiver_city','Delivered_By_Hub_Code','0'], inplace=True)
        after_drop = len(data)
        print(f"[INFO] Dropped {before_drop - after_drop} rows with NA in essential columns")

        # Remove duplicates
        before_dup = len(data)
        data.drop_duplicates(subset=['address','Delivered_By_Hub_Code'], inplace=True)
        after_dup = len(data)
        print(f"[INFO] Dropped {before_dup - after_dup} duplicate address-target rows")

        # cleaning V series
        data = clean_V_cp(data)

        # Validation
        data['is_valid'] = data['address'].apply(clean_and_validate_address)
        print(f"[INFO] Completed validation on addresses")

        # Final feature column
        data['full_address'] = data['address'] + ' ' + data['foc_receiver_pincode'] + ' ' + data['foc_receiver_city']
        
        # Set invalid targets to INVALID
        invalid_count = (~data['is_valid']).sum()
        print(f"[INFO] Marked {invalid_count} rows as INVALID")

        # Grouping low-sample targets
        print(f"[INFO] Finished processing. Final rows: {len(data)}")

        return data
    except Exception as e:
        print(f"[ERROR] data_preprocess failed, error: {e}")
        return data

124


In [1]:
import pandas as pd

In [2]:

df = pd.read_parquet('/Users/akash/Documents/cp_pred_github/cp-prediction-service/notebook/pan_India_test_data_2025-07-23.parquet')
df.head()

,foc_booking_date,foc_bkdocument_code,foc_Booked_Wt,foc_booking_app,foc_receiver_address,foc_receiver_city,foc_receiver_state,foc_receiver_pincode,Shipment_ID,Delivered_By_Hub_Type,...,address_len,0,v_series,is_valid,full_address,pincode_first2,prediction_invalid,prediction_invalid_prob,invalid_version,cp_code
0,2025-07-23,DOX,0.020,APP,"Rajan Thottam , Thanjavur Dist, kumbakonam -61...",thanjavur,TAMIL NADU,612001,C46728703,FR,...,39,EF1114,0,False,rajan thottam thanjavur dist kumbakonam 612001...,61,INVALID,0.293002,ggn_v1,INVALID
1,2025-07-23,NDX,0.240,FM,146 WEST STREET ADALAIYUR ENANGUDI NAGAPPATINA...,thanjavur,TAMIL NADU,613001,7X106431388,FR,...,86,EF326,0,True,146 west street adalaiyur egudi nagappatinam d...,61,OTHER_GURGAON,0.064833,ggn_v1,EF326
2,2025-07-23,NDX,0.470,APP,than,thanjavur,TAMIL NADU,613007,U62504736,FR,...,4,EF150,0,False,than 613007 thanjavur,61,INVALID,0.751287,ggn_v1,INVALID
3,2025-07-23,NDX,0.940,APP,59/1 voc street pallavermedu west,thanjavur,TAMIL NADU,614205,U72904676,FR,...,33,EF1716,0,True,59 1 voc street pallavermedu west 614205 thanj...,61,OTHER_GURGAON,0.082897,ggn_v1,EF1716
4,2025-07-23,NDX,1.377,APP,ayyan,thanjavur,TAMIL NADU,614201,7D115648470,FR,...,5,EF1914,0,False,ayyan 614201 thanjavur,61,INVALID,0.783381,ggn_v1,INVALID


In [1]:
df = pd.read_csv('data/merged_data_2025-08-06.csv')
print(df.shape)
df1=pd.read_csv('data/processed/Processed_merged_data_2025-08-06.csv')
print(df1[df1['is_valid']==False].shape)

NameError: name 'pd' is not defined

In [55]:
df.head()

,foc_booking_date,foc_bkdocument_code,foc_Booked_Wt,foc_booking_app,foc_receiver_address,foc_receiver_city,foc_receiver_state,foc_receiver_pincode,Shipment_ID,Delivered_By_Hub_Type,Delivered_By_Hub_Code
0,2025-08-04,NDX,1.00,EBK,"16-7-761 CHADERGHAT SAHIFA, ATTARI APPARTMENT",HYDERABAD,TELANGANA,500024,7X150284638,FR,HF1293
1,2025-08-04,DOX,0.10,EBK,CHERLAI,COCHIN,KERALA,682002,D1003268570,FR,OF822
2,2025-08-04,DOX,0.20,EBK,"Hayathnagar, hydrabad",KOLHAPUR,MAHARASHTRA,416122,P84948812,FR,PF1796
3,2025-08-04,DOX,0.10,EBK,PATHADIPALAM,COCHIN,KERALA,682002,D1003268550,FR,OF822
4,2025-08-04,DOX,0.10,EBK,PANAYAPPILY,COCHIN,KERALA,682002,D1003268551,FR,OF822


In [56]:
df1.head()

,foc_booking_date,foc_bkdocument_code,foc_Booked_Wt,foc_booking_app,foc_receiver_address,foc_receiver_city,foc_receiver_state,foc_receiver_pincode,Shipment_ID,Delivered_By_Hub_Type,Delivered_By_Hub_Code,address,address_len,0,v_series,is_valid,full_address
0,2025-08-04,NDX,1.00,EBK,"16-7-761 CHADERGHAT SAHIFA, ATTARI APPARTMENT",hyderabad,TELANGANA,500024,7X150284638,FR,HF1293,16 7 761 chaderghat sahifa attari appartment,44,HF1293,0,True,16 7 761 chaderghat sahifa attari appartment 500024 hyderabad
1,2025-08-04,DOX,0.10,EBK,CHERLAI,cochin,KERALA,682002,D1003268570,FR,OF822,cherlai,7,OF822,0,False,cherlai 682002 cochin
2,2025-08-04,DOX,0.20,EBK,"Hayathnagar, hydrabad",kolhapur,MAHARASHTRA,416122,P84948812,FR,PF1796,hayathnagar hydrabad,20,PF1796,0,True,hayathnagar hydrabad 416122 kolhapur
3,2025-08-04,DOX,0.10,EBK,PATHADIPALAM,cochin,KERALA,682002,D1003268550,FR,OF822,pathadipalam,12,OF822,0,False,pathadipalam 682002 cochin
4,2025-08-04,DOX,0.10,EBK,PANAYAPPILY,cochin,KERALA,682002,D1003268551,FR,OF822,panayappily,11,OF822,0,False,panayappily 682002 cochin


In [4]:

# df1 = data_preprocess(df)

# Preprocessing on all files

In [5]:
import os
from tqdm import tqdm

def process_all_files(input_folder, output_folder):
    """
    Reads every CSV file from input_folder, applies data_preprocess, and saves the output DataFrame to output_folder.
    Output files are saved with the same filename as input.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    csv_filenames = [f for f in os.listdir(input_folder) if f.endswith('.csv')]
    for filename in tqdm(csv_filenames, desc="Processing CSV files"):
        input_path = os.path.join(input_folder, filename)
        try:
            df = pd.read_csv(input_path)
            processed_df = data_preprocess(df)
            output_path = os.path.join(output_folder, f'Processed_{filename}')
            processed_df.to_csv(output_path, index=False)
            print(f"[INFO] Processed and saved: {output_path}")
        except Exception as e:
            print(f"[ERROR] Failed to process {filename}: {e}")

# Example usage:
process_all_files('data', 'data/processed')


Processing CSV files:   0%|          | 0/97 [00:00<?, ?it/s]

[INFO] Starting preprocessing...
[INFO] Dropped 68938 rows with NA in essential columns
[INFO] Dropped 44488 duplicate address-target rows
[INFO] Completed validation on addresses
[INFO] Marked 88402 rows as INVALID
[INFO] Finished processing. Final rows: 528028


Processing CSV files:   1%|          | 1/97 [00:07<12:24,  7.75s/it]

[INFO] Processed and saved: data/processed/Processed_merged_data_2025-07-22.csv
[INFO] Starting preprocessing...
[INFO] Dropped 76967 rows with NA in essential columns
[INFO] Dropped 101931 duplicate address-target rows
[INFO] Completed validation on addresses
[INFO] Marked 201806 rows as INVALID
[INFO] Finished processing. Final rows: 377754


Processing CSV files:   2%|▏         | 2/97 [00:12<09:53,  6.24s/it]

...

[INFO] Dropped 16354 rows with NA in essential columns
[INFO] Dropped 4222 duplicate address-target rows
[INFO] Completed validation on addresses
[INFO] Marked 11977 rows as INVALID
[INFO] Finished processing. Final rows: 56817


Processing CSV files:  99%|█████████▉| 96/97 [07:42<00:03,  3.52s/it]

[INFO] Processed and saved: data/processed/Processed_merged_data_2025-07-13.csv
[INFO] Starting preprocessing...
[INFO] Dropped 54038 rows with NA in essential columns
[INFO] Dropped 36717 duplicate address-target rows
[INFO] Completed validation on addresses
[INFO] Marked 68082 rows as INVALID
[INFO] Finished processing. Final rows: 449392


Processing CSV files: 100%|██████████| 97/97 [07:48<00:00,  4.83s/it]

[INFO] Processed and saved: data/processed/Processed_merged_data_2025-07-07.csv


In [6]:
import glob

csv_files = glob.glob("data/processed/*.csv")
print(f"Number of CSV files in the folder: {len(csv_files)}")


Number of CSV files in the folder: 97


# reading all processed files

In [7]:
df=pd.read_csv('data/processed/Processed_merged_data_2025-05-01.csv')

In [9]:
df[df['v_series']==1].head()

,foc_booking_date,foc_bkdocument_code,foc_Booked_Wt,foc_booking_app,foc_receiver_address,foc_receiver_city,foc_receiver_state,foc_receiver_pincode,Shipment_ID,Delivered_By_Hub_Type,Delivered_By_Hub_Code,address,address_len,0,v_series,is_valid,full_address
10,2025-05-01,NDX,0.22,APP,xyz,dhubri,ASSAM,783339,D2001242217,FR,XV00005,xyz,3,XV00005,1,False,xyz 783339 dhubri
12,2025-05-01,NDX,0.19,APP,xyz,chennai,TAMIL NADU,603112,D2001242170,FR,CV00005,xyz,3,CV00005,1,False,xyz 603112 chennai
56,2025-05-01,NDX,7.00,FM,"International Tobacco Co- Ltd- Godfrey Phillips Ltd, Meerut Road Industrial Area, Ghaziabad, Uttar Pradesh 201003",ghaziabad,UTTAR PRADESH,201003,X50673148,FR,UV00196,international tobacco co ltd godfrey phillips ltd meerut road industrial area ghaziabad uttar pradesh,101,UV00196,1,True,international tobacco co ltd godfrey phillips ltd meerut road industrial area ghaziabad uttar pradesh 201003 ghaziabad
74,2025-05-01,NDX,14.99,APP,sss,alwar,RAJASTHAN,301001,D3000205090,FR,QV00019,sss,3,QV00019,1,False,sss 301001 alwar
80,2025-05-01,NDX,0.15,APP,xyz,gurgaon,HARYANA,122018,W61713960,FR,LV00011,xyz,3,LV00011,1,False,xyz 122018 gurgaon


In [10]:
import glob

# Read all CSV files in the folder into a list of DataFrames
csv_files = glob.glob("data/processed/*.csv")
dfs = [pd.read_csv(f) for f in csv_files]

# Fake 1 dataframe by concatenating all DataFrames (as an example)
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    print("Combined DataFrame shape:", combined_df.shape)
    combined_df.head()
else:
    print("No CSV files found in the folder.")


# Save the combined DataFrame as a parquet file
combined_df['foc_receiver_pincode'] = combined_df['foc_receiver_pincode'].astype(str)
combined_df.to_parquet("data/input/combined_data.parquet", index=False)


/var/folders/bp/x07bnl_d2p76ck3mkv847qrc0000gn/T/ipykernel_27367/578718936.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, ignore_index=True)


Combined DataFrame shape: (32349930, 17)


In [11]:

combined_df.shape

(32349930, 17)

In [12]:
combined_df.head()

,foc_booking_date,foc_bkdocument_code,foc_Booked_Wt,foc_booking_app,foc_receiver_address,foc_receiver_city,foc_receiver_state,foc_receiver_pincode,Shipment_ID,Delivered_By_Hub_Type,Delivered_By_Hub_Code,address,address_len,0,v_series,is_valid,full_address
0,2025-06-14,NDX,1.71,FM,Flat no B 106 New Three Room Near ME School Ground Noamundi Apartment,jamshedpur,JHARKHAND,833217,I76747731,FR,TF412,flat no b 106 new three room near me school ground noamundi apartment,69,TF412,0,True,flat no b 106 new three room near me school ground noamundi apartment 833217 jamshedpur
1,2025-06-14,NDX,1.65,FM,kasaragod kanhangad,kasargod,KERALA,671531,7D113956759,FR,OF1353,kasaragod kanhangad,19,OF1353,0,False,kasaragod kanhangad 671531 kasargod
2,2025-06-14,NDX,1.05,FM,Aviral sadan singharia near family super mart Kunraghat,gorakhpur,UTTAR PRADESH,273008,7D111537424,FR,UF842,aviral sadan singharia near family super mart kunraghat,55,UF842,0,True,aviral sadan singharia near family super mart kunraghat 273008 gorakhpur
3,2025-06-14,NDX,0.50,FM,"Devine library near Hansi gate, Bhiwani",rohtak,HARYANA,127021,7D110672437,FR,JF481,devine library near hansi gate bhiwani,38,JF481,0,True,devine library near hansi gate bhiwani 127021 rohtak
4,2025-06-14,NDX,12.00,FM,"C/O-Delhivery Pvt. Ltd. ,Khasra No-347, Bhovapur,Reliance Road, Jindal Ngr,Hapur",noida,UTTAR PRADESH,201301,D1002034354,BR,N30,c o delhivery pvt ltd khasra no 347 bhovapur reliance road jindal ngr hapur,75,N30,0,True,c o delhivery pvt ltd khasra no 347 bhovapur reliance road jindal ngr hapur 201301 noida


In [2]:
parquet_path = "data/input/combined_data.parquet"
combined_df = pd.read_parquet(parquet_path)

In [3]:
combined_df.head()

,foc_booking_date,foc_bkdocument_code,foc_Booked_Wt,foc_booking_app,foc_receiver_address,foc_receiver_city,foc_receiver_state,foc_receiver_pincode,Shipment_ID,Delivered_By_Hub_Type,Delivered_By_Hub_Code,address,address_len,0,v_series,is_valid,full_address,pincode_first2
0,2025-06-14,NDX,1.71,FM,Flat no B 106 New Three Room Near ME School Ground Noamundi Apartment,jamshedpur,JHARKHAND,833217,I76747731,FR,TF412,flat no b 106 new three room near me school ground noamundi apartment,69,TF412,0,True,flat no b 106 new three room near me school ground noamundi apartment 833217 jamshedpur,83
1,2025-06-14,NDX,1.65,FM,kasaragod kanhangad,kasargod,KERALA,671531,7D113956759,FR,OF1353,kasaragod kanhangad,19,OF1353,0,False,kasaragod kanhangad 671531 kasargod,67
2,2025-06-14,NDX,1.05,FM,Aviral sadan singharia near family super mart Kunraghat,gorakhpur,UTTAR PRADESH,273008,7D111537424,FR,UF842,aviral sadan singharia near family super mart kunraghat,55,UF842,0,True,aviral sadan singharia near family super mart kunraghat 273008 gorakhpur,27
3,2025-06-14,NDX,0.50,FM,"Devine library near Hansi gate, Bhiwani",rohtak,HARYANA,127021,7D110672437,FR,JF481,devine library near hansi gate bhiwani,38,JF481,0,True,devine library near hansi gate bhiwani 127021 rohtak,12
4,2025-06-14,NDX,12.00,FM,"C/O-Delhivery Pvt. Ltd. ,Khasra No-347, Bhovapur,Reliance Road, Jindal Ngr,Hapur",noida,UTTAR PRADESH,201301,D1002034354,BR,N30,c o delhivery pvt ltd khasra no 347 bhovapur reliance road jindal ngr hapur,75,N30,0,True,c o delhivery pvt ltd khasra no 347 bhovapur reliance road jindal ngr hapur 201301 noida,20


In [4]:
combined_df.shape

(32349930, 18)

In [5]:
combined_df['foc_receiver_pincode'].isna().sum()

np.int64(0)

In [6]:
# Extract first 2 digits of foc_receiver_pincode and make a new column
combined_df['pincode_first2'] = combined_df['foc_receiver_pincode'].str[:2]


In [7]:
combined_df['pincode_first2'].nunique()

69

In [ ]:
combined_df['pincode_first2'].value_counts()


pincode_first2
50    2454261
40    2197186
56    2185596
11    1958572
41    1460816
68    1431790
60    1278472
12    1269654
67    1116906
20    1114936
70     863273
14     819106
52     756506
53     709381
69     574850
24     558337
38     542584
57     518530
30     516544
22     482515
51     477652
78     469860
13     458535
42     426968
63     408028
64     407615
39     400272
62     385205
58     380173
71     354857
44     333882
75     330250
45     283457
49     241598
16     215953
28     201691
73     200500
17     191290
83     178560
74     166363
76     166264
46     165542
27     159431
80     152139
43     147788
36     146544
72     136601
26     135867
25     133038
82     120407
33     117397
48     115892
31     112398
84     110432
15     109995
21     108011
79     104512
47      98536
18      96686
34      94600
32      91419
59      67877
19      65651
81      56299
85      54945
23      50454
61      49894
37      48681
77      10106
Name: count, dtype:

In [24]:
path = output_folder = "data/input/pincode_first2_wise_invalid_pred/32_data_invalid_pred.parquet"
df=pd.read_parquet(path)
df.shape

(91419, 21)

In [25]:
df.head()

,foc_booking_date,foc_bkdocument_code,foc_Booked_Wt,foc_booking_app,foc_receiver_address,foc_receiver_city,foc_receiver_state,foc_receiver_pincode,Shipment_ID,Delivered_By_Hub_Type,Delivered_By_Hub_Code,address,address_len,0,v_series,is_valid,full_address,pincode_first2,prediction_invalid,prediction_invalid_prob,invalid_version
0,2025-06-14,NDX,3.95,APP,kota,kota,RAJASTHAN,324005,D3001257156,FR,QF083,kota,4,QF083,0,False,kota 324005 kota,32,INVALID,0.78,ggn_v1
1,2025-06-14,DOX,0.02,APP,"hshhs, null, null",kota,RAJASTHAN,323001,Z61285087,FR,QF080,hshhs,5,QF080,0,False,hshhs 323001 kota,32,INVALID,0.88,ggn_v1
2,2025-06-14,NDX,3.00,EBK,"KOTA, KOTA",kota,RAJASTHAN,326502,D1003266425,FR,QF1277,kota kota,9,QF1277,0,False,kota kota 326502 kota,32,INVALID,0.52,ggn_v1
3,2025-06-14,NDX,3.30,EBK,"KOTA, KOTA",kota,RAJASTHAN,324010,D1003266489,FR,QF079,kota kota,9,QF079,0,False,kota kota 324010 kota,32,INVALID,0.79,ggn_v1
4,2025-06-14,NDX,3.00,EBK,"JAIPUR, JAIPUR",jaipur,RAJASTHAN,322001,D1003266520,FR,QV00194,jaipur jaipur,13,QV00194,1,False,jaipur jaipur 322001 jaipur,32,INVALID,0.92,ggn_v1


In [ ]:
df['foc_receiver_pincode'].value_counts()


foc_receiver_pincode
324005    13811
321001     8229
324001     7077
324007     5781
324009     4355
327001     4148
324002     3630
323001     3187
325205     2989
324008     2980
326001     2558
328001     2432
324006     2298
324010     2266
322001     1949
322230     1635
326502     1331
323307     1329
322201     1251
326519     1176
326023     1164
324004     1031
325220      951
321203      787
324003      658
326033      621
328021      618
321022      610
321401      581
325001      568
321608      558
325003      494
327024      430
325202      412
327022      382
326038      379
325221      367
327023      330
323603      330
325216      309
325219      292
325218      264
322241      196
325204      181
326520      174
327034      171
325215      160
322021      152
327025      148
325201      137
321205      120
321602      111
321606       94
326517       89
326022       84
323305       79
323301       72
326530       71
323306       65
321303       64
323021       64
321

In [11]:
combined_df.columns

Index(['foc_booking_date', 'foc_bkdocument_code', 'foc_Booked_Wt',
       'foc_booking_app', 'foc_receiver_address', 'foc_receiver_city',
       'foc_receiver_state', 'foc_receiver_pincode', 'Shipment_ID',
       'Delivered_By_Hub_Type', 'Delivered_By_Hub_Code', 'address',
       'address_len', '0', 'v_series', 'is_valid', 'full_address',
       'pincode_first2'],
      dtype='object')

In [ ]:
combined_df[combined_df['foc_receiver_city'].str.lower().str.startswith('gur')]['foc_receiver_city'].unique()



In [20]:
combined_df[combined_df['foc_receiver_city']=='delhi'].shape

(1958531, 17)

In [22]:
# Find city wise unique Delivered_By_Hub_Code count and total count per city
city_hub_unique_counts = (
    combined_df.groupby('foc_receiver_city')
    .agg(
        unique_hub_code_count=('Delivered_By_Hub_Code', 'nunique'),
        shipments_count=('Delivered_By_Hub_Code', 'count')
    )
    .reset_index()
)
city_hub_unique_counts=city_hub_unique_counts.sort_values(by='shipments_count',ascending=0).reset_index(drop=True)

In [24]:
city_hub_unique_counts['shipments_count'].sum()

np.int64(32349930)

In [25]:
city_hub_unique_counts['cumulative_percentage'] = city_hub_unique_counts['shipments_count'].cumsum() / city_hub_unique_counts['shipments_count'].sum() * 100


In [26]:
city_hub_unique_counts.head(100)

,foc_receiver_city,unique_hub_code_count,shipments_count,cumulative_percentage
0,bangalore,491,2176164,6.73
1,mumbai,444,1993476,12.89
2,delhi,696,1958531,18.94
3,hyderabad,547,1866422,24.71
4,chennai,345,1047532,27.95
5,pune,329,995621,31.03
6,kolkata,336,876778,33.74
7,gurgaon,233,679335,35.84
8,noida,206,535579,37.49
9,cochin,130,517970,39.10


In [ ]:
# city_hub_unique_counts.to_csv('data/input/city_wise_cps_shipments.csv',index=False)

In [5]:
# city_hub_unique_counts

In [29]:
import json

top_100_cities = city_hub_unique_counts['foc_receiver_city'].head(100).tolist()

# Save top_100_cities to a JSON file
with open('data/input/top_100_cities.json', 'w') as f:
    json.dump(top_100_cities, f, indent=2)

top_100_cities


['bangalore',
 'mumbai',
 'delhi',
 'hyderabad',
 'chennai',
 'pune',
 'kolkata',
 'gurgaon',
 'noida',
 'cochin',
 'ahmedabad',
 'jaipur',
 'vijayawada',
 'trivandrum',
 'trichur',
 'coimbatore',
 'lucknow',
 'kottakkal',
 'vishakapatnam',
 'calicut',
 'guntur',
 'ludhiana',
 'nagpur',
 'indore',
 'quilon',
 'guwahati',
 'kanpur',
 'palghat',
 'kottayam',
 'patna',
 'ghaziabad',
 'varanasi',
 'ambala',
 'surat',
 'nasik',
 'cannanore',
 'chandigarh',
 'rajahmundry',
 'kalyan',
 'faridabad',
 'bhubaneshwar',
 'bhopal',
 'salem',
 'madurai',
 'dehradun',
 'agra',
 'baroda',
 'jalandhar',
 'panchkula',
 'raipur',
 'panvel',
 'mysore',
 'tiruvalla',
 'ranchi',
 'alleppey',
 'trichy',
 'serampore',
 'howrah',
 'hubli',
 'rajkot',
 'ballabgarh',
 'tirupati',
 'nellore',
 'ongole',
 'mangalore',
 'bhatinda',
 'amritsar',
 'kurnool',
 'khammam',
 'allahabad',
 'meerut',
 'gorakhpur',
 'vaishali up',
 'jammu',
 'ananthpur',
 'aurangabad mh',
 'muvattupuzha',
 'kundli',
 'warangal',
 'vasai',
 

# making city wise data

In [6]:
with open('data/input/top_100_cities.json', 'r') as f:
    top_100_cities = json.load(f)


import os
from tqdm import tqdm

output_dir = "data/input/city_wise"
os.makedirs(output_dir, exist_ok=True)

for city in tqdm(top_100_cities, desc="Saving city-wise parquet files"):
    city_df = combined_df[combined_df['foc_receiver_city'] == city]
    output_path = os.path.join(output_dir, f"{city}_data.parquet")
    city_df.to_parquet(output_path, index=False)


python(30914) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Saving city-wise parquet files: 100%|██████████| 100/100 [14:20<00:00,  8.60s/it]


# pincode_first2 wise data

In [5]:
# Get all unique pincode_first2 values
pincode_first2_list = sorted(combined_df['pincode_first2'].dropna().unique())
print(f"📊 Processing {len(pincode_first2_list)} unique pincode_first2 values")
print(f"📁 Total rows in dataset: {len(combined_df):,}")
    
    # Use pandas groupby - this is the fastest method
    # It avoids repeated filtering operations on the entire DataFrame
processed_count = 0
total_rows = 0
    
    # Create the groupby object first
grouped_data = combined_df.groupby('pincode_first2')

📊 Processing 69 unique pincode_first2 values
📁 Total rows in dataset: 32,349,930


In [20]:
# grouped_data.head()

In [6]:
import time

start_time = time.time()

output_dir = "data/input/pincode_first2_wise"
os.makedirs(output_dir, exist_ok=True)
    

    
# Iterate through each group
for pincode_first2, group_df in tqdm(grouped_data, 
                                         desc="🚀 Processing pincode files",
                                         total=len(pincode_first2_list)):
        
        # Save the group directly - no additional filtering needed
        output_path = os.path.join(output_dir, f"{pincode_first2}_data.parquet")
        group_df.to_parquet(output_path, index=False)
        
        processed_count += 1
        total_rows += len(group_df)
        
        # Progress update every 10 files
        if processed_count % 10 == 0:
            elapsed = time.time() - start_time
            rate = processed_count / elapsed
            eta = (len(pincode_first2_list) - processed_count) / rate if rate > 0 else 0
            print(f"  📈 Progress: {processed_count}/{len(pincode_first2_list)} files, "
                  f"Rate: {rate:.1f} files/sec, ETA: {eta:.1f}s")

total_time = time.time() - start_time

🚀 Processing pincode files:  14%|█▍        | 10/69 [01:15<02:32,  2.58s/it] 

  📈 Progress: 10/69 files, Rate: 0.1 files/sec, ETA: 446.0s


🚀 Processing pincode files:  29%|██▉       | 20/69 [01:30<01:15,  1.54s/it]

  📈 Progress: 20/69 files, Rate: 0.2 files/sec, ETA: 221.6s


🚀 Processing pincode files:  43%|████▎     | 30/69 [01:50<01:52,  2.89s/it]

...

🚀 Processing pincode files:  72%|███████▏  | 50/69 [02:37<00:41,  2.16s/it]

  📈 Progress: 50/69 files, Rate: 0.3 files/sec, ETA: 59.9s


🚀 Processing pincode files:  88%|████████▊ | 61/69 [03:01<00:10,  1.36s/it]

  📈 Progress: 60/69 files, Rate: 0.3 files/sec, ETA: 27.2s


🚀 Processing pincode files: 100%|██████████| 69/69 [03:11<00:00,  2.78s/it]


In [21]:
df=pd.read_parquet('data/input/pincode_first2_wise/40_data.parquet')

In [22]:
df.shape

(2197186, 18)

In [23]:
df.head()

,foc_booking_date,foc_bkdocument_code,foc_Booked_Wt,foc_booking_app,foc_receiver_address,foc_receiver_city,foc_receiver_state,foc_receiver_pincode,Shipment_ID,Delivered_By_Hub_Type,Delivered_By_Hub_Code,address,address_len,0,v_series,is_valid,full_address,pincode_first2
0,2025-06-14,NDX,0.05,FM,"Shree Sai Ganesh Apartment Plot number 146 room no 106, Turbhe sector 22",mumbai,MAHARASHTRA,400705,7D117191770,FR,MF1053,shree sai ganesh apartment plot number 146 room no 106 turbhe sector 22,71,MF1053,0,True,shree sai ganesh apartment plot number 146 room no 106 turbhe sector 22 400705 mumbai,40
1,2025-06-14,NDX,0.90,FM,304 damar galli 51 57 opp raza acadamy khadak mumbai,mumbai,MAHARASHTRA,400003,7D114526306,FR,MF1462,304 damar galli 51 57 opp raza acadamy khadak mumbai,52,MF1462,0,True,304 damar galli 51 57 opp raza acadamy khadak mumbai 400003 mumbai,40
2,2025-06-14,NDX,0.24,FM,"Dasgaon, Taluka Mahad District Raigad, Dasgaon, Taluka Mahad District Raigad,",panvel,MAHARASHTRA,402301,I76911339,FR,MF2563,dasgaon taluka mahad district raigad dasgaon taluka mahad district raigad,73,MF2563,0,True,dasgaon taluka mahad district raigad dasgaon taluka mahad district raigad 402301 panvel,40
3,2025-06-14,NDX,0.40,FM,"D & M Enterprises 109/110, Sai Jeevan, S.V.P.Road, Opp. Mulund Railway Station,",mumbai,MAHARASHTRA,400080,V98585780,FR,MF1016,d m enterprises 109 110 sai jeevan s v p road opp mulund railway station,72,MF1016,0,True,d m enterprises 109 110 sai jeevan s v p road opp mulund railway station 400080 mumbai,40
4,2025-06-14,NDX,0.50,FM,Lodha Splendora Ghodbunder Road Bhayandarpada Thane West,mumbai,MAHARASHTRA,400615,7D114681500,FR,MF2297,lodha splendora ghodbunder road bhayandarpada thane west,56,MF2297,0,True,lodha splendora ghodbunder road bhayandarpada thane west 400615 mumbai,40


In [ ]:
# df.shape

(2454261, 18)

In [20]:
unique_dates = df['foc_booking_date'].drop_duplicates().sort_values(ascending=True)
unique_dates

1329238    2025-05-01
1554319    2025-05-02
1633817    2025-05-03
1156078    2025-05-04
1310005    2025-05-05
1114210    2025-05-06
979917     2025-05-07
651029     2025-05-08
842030     2025-05-09
1159042    2025-05-10
1307566    2025-05-11
1136962    2025-05-12
959970     2025-05-13
1514761    2025-05-14
1342617    2025-05-15
1535353    2025-05-16
1653727    2025-05-17
626207     2025-05-18
494270     2025-05-19
629202     2025-05-20
860776     2025-05-21
938919     2025-05-22
881484     2025-05-23
607247     2025-05-24
505738     2025-05-25
399214     2025-05-26
473545     2025-05-27
1491710    2025-05-28
1363458    2025-05-29
587414     2025-05-30
51915      2025-06-01
101409     2025-06-02
78683      2025-06-03
346679     2025-06-04
270735     2025-06-05
179904     2025-06-06
200910     2025-06-07
1723627    2025-06-08
1700240    2025-06-09
319085     2025-06-10
293464     2025-06-11
149872     2025-06-12
219975     2025-06-13
0          2025-06-14
49628      2025-06-15
121354    

# train / test sepration

In [32]:
import pandas as pd
import glob
import os
from tqdm import tqdm

city_wise_dir = "data/input/pincode_first2_wise_invalid_pred_filtered/"
train_dir = "data/input/pincode_first2_wise_train/"
test_dir = "data/input/pincode_first2_wise_test/"
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

split_date = "2025-07-21"

parquet_files = glob.glob(os.path.join(city_wise_dir, "*.parquet"))

for file_path in tqdm(parquet_files, desc="Splitting city parquet files"):
    city_name = os.path.basename(file_path).replace("_data_invalid_pred.parquet", "")
    df_city = pd.read_parquet(file_path)
    # Ensure foc_booking_date is string for comparison
    df_city['foc_booking_date'] = df_city['foc_booking_date'].astype(str)
    train_df = df_city[df_city['foc_booking_date'] < split_date]
    test_df = df_city[df_city['foc_booking_date'] >= split_date]
    train_path = os.path.join(train_dir, f"{city_name}_data_train.parquet")
    test_path = os.path.join(test_dir, f"{city_name}_data_test.parquet")
    train_df.to_parquet(train_path, index=False)
    test_df.to_parquet(test_path, index=False)


Splitting city parquet files:   0%|          | 0/69 [00:00<?, ?it/s]

Splitting city parquet files: 100%|██████████| 69/69 [01:06<00:00,  1.05it/s]


In [6]:
df1=pd.read_parquet('data/input/pincode_first2_wise_train/40_data_train.parquet')
df2=pd.read_parquet('data/input/pincode_first2_wise_test/40_data_test.parquet')
print(df1.shape,df2.shape)
print(df1.shape[0]+df2.shape[0])

(1828102, 22) (369084, 22)
2197186


In [7]:
df2.head()

,foc_booking_date,foc_bkdocument_code,foc_Booked_Wt,foc_booking_app,foc_receiver_address,foc_receiver_city,foc_receiver_state,foc_receiver_pincode,Shipment_ID,Delivered_By_Hub_Type,Delivered_By_Hub_Code,address,address_len,0,v_series,is_valid,full_address,pincode_first2,prediction_invalid,prediction_invalid_prob,invalid_version,cp_code
0,2025-08-04,DOX,0.02,APP,nnj,mumbai,MAHARASHTRA,400025,T31632416,FR,MF1131,nnj,3,MF1131,0,False,nnj 400025 mumbai,40,INVALID,0.93,ggn_v1,INVALID
1,2025-08-04,DOX,0.10,APP,"201, 2nd floor, Dasti pinnacle, Above New passport office, Road No. 22, wagle Estate,",mumbai,MAHARASHTRA,400604,M56030490,FR,MF786,201 2nd floor dasti pinnacle above new passport office road no 22 wagle estate,78,MF786,0,True,201 2nd floor dasti pinnacle above new passport office road no 22 wagle estate 400604 mumbai,40,LF632,0.05,ggn_v1,MF786
2,2025-08-04,DOX,0.10,APP,"142-S48, S.V. Road, Jogeshwari (W), Mumbai-400102",mumbai,MAHARASHTRA,400102,M51489359,FR,MF1851,142 s48 s v road jogeshwari w mumbai,36,MF1851,0,True,142 s48 s v road jogeshwari w mumbai 400102 mumbai,40,INVALID,0.20,ggn_v1,MF1851
3,2025-08-04,DOX,0.01,APP,tardeo,mumbai,MAHARASHTRA,400034,M51496004,FR,MF2334,tardeo,6,MF2334,0,False,tardeo 400034 mumbai,40,INVALID,0.77,ggn_v1,INVALID
4,2025-08-04,DOX,0.10,APP,"184-187 S.V. Road, Jogeshwari (W), Mumbai, India - 400102",mumbai,MAHARASHTRA,400102,M51489363,FR,MF1851,184 187 s v road jogeshwari w mumbai india,42,MF1851,0,True,184 187 s v road jogeshwari w mumbai india 400102 mumbai,40,INVALID,0.14,ggn_v1,MF1851


# pincode wise common cp

In [ ]:
parquet_path = "data/input/combined_data.parquet"
combined_df = pd.read_parquet(parquet_path)

In [71]:
# Get the most common Delivered_By_Hub_Code in each pincode and its percentage
result = (
    combined_df.groupby('foc_receiver_pincode')['Delivered_By_Hub_Code']
    .value_counts(normalize=True)
    .groupby(level=0)
    .head(1)
    .reset_index(name='percentage')
)
result = result.rename(columns={'Delivered_By_Hub_Code': 'most_common_hub_code'})
result['percentage'] = (result['percentage'] * 100).round(2)

In [73]:
result['foc_receiver_pincode'].nunique()

13400

In [75]:
result.to_csv('data/input/pincode_wise_common_cp.csv',index=False)

In [72]:
result.head()

,foc_receiver_pincode,most_common_hub_code,percentage
0,110001,NV099,19.24
1,110002,NF087,25.77
2,110003,SF158,22.70
3,110004,N10,44.85
4,110005,N10,15.24


In [5]:
combined_df['pincode_first2'].value_counts().reset_index()


,pincode_first2,count
0,50,2454261
1,40,2197186
2,56,2185596
3,11,1958572
4,41,1460816
5,68,1431790
6,60,1278472
7,12,1269654
8,67,1116906
9,20,1114936


# csv counting

In [31]:
import glob

csv_files = glob.glob("data/input/pincode_first2_wise_invalid_pred/*.parquet")
print(f"Number of CSV files in the folder: {len(csv_files)}")


Number of CSV files in the folder: 69


In [4]:
parquet_path = "data/input/combined_data.parquet"
df = pd.read_parquet(parquet_path)

In [5]:
df.head()

,foc_booking_date,foc_bkdocument_code,foc_Booked_Wt,foc_booking_app,foc_receiver_address,foc_receiver_city,foc_receiver_state,foc_receiver_pincode,Shipment_ID,Delivered_By_Hub_Type,Delivered_By_Hub_Code,address,address_len,is_valid,full_address
0,2025-06-14,NDX,1.71,FM,Flat no B 106 New Three Room Near ME School Gr...,jamshedpur,JHARKHAND,833217,I76747731,FR,TF412,flat no b 106 new three room near me school gr...,69,True,flat no b 106 new three room near me school gr...
1,2025-06-14,NDX,1.65,FM,kasaragod kanhangad,kasargod,KERALA,671531,7D113956759,FR,OF1353,kasaragod kanhangad,19,False,kasaragod kanhangad 671531 kasargod
2,2025-06-14,NDX,1.05,FM,Aviral sadan singharia near family super mart ...,gorakhpur,UTTAR PRADESH,273008,7D111537424,FR,UF842,aviral sadan singharia near family super mart ...,55,True,aviral sadan singharia near family super mart ...
3,2025-06-14,NDX,0.50,FM,"Devine library near Hansi gate, Bhiwani",rohtak,HARYANA,127021,7D110672437,FR,JF481,devine library near hansi gate bhiwani,38,True,devine library near hansi gate bhiwani 127021 ...
4,2025-06-14,NDX,12.00,FM,"C/O-Delhivery Pvt. Ltd. ,Khasra No-347, Bhovap...",noida,UTTAR PRADESH,201301,D1002034354,BR,N30,c o delhivery pvt ltd khasra no 347 bhovapur r...,75,True,c o delhivery pvt ltd khasra no 347 bhovapur r...


In [9]:
# Aggregate on foc_receiver_city and find Delivered_By_Hub_Code where 2nd character is 'V'

# Filter rows where the 2nd character of Delivered_By_Hub_Code is 'V'
filtered_df = df[df['Delivered_By_Hub_Code'].astype(str).str[1] == 'V']

# Aggregate: get list of such Delivered_By_Hub_Code per foc_receiver_city
agg_df = filtered_df.groupby('foc_receiver_city')['Delivered_By_Hub_Code'].apply(lambda x: list(set(x))).reset_index()

# Add a column for the length of the list
agg_df['list_length'] = agg_df['Delivered_By_Hub_Code'].apply(len)

# Sort in descending order of list length
agg_df = agg_df.sort_values(by='list_length', ascending=False).reset_index(drop=True)

agg_df.head()


,foc_receiver_city,Delivered_By_Hub_Code,list_length
0,chennai,"[CV00145, CV00098, CV00153, CV00014, CV00101, ...",31
1,mumbai,"[MV00277, MV00134, MV00125, MV00254, MV00249, ...",30
2,kolkata,"[KV00188, KV00185, KV00013, KV00098, KV00081, ...",26
3,ahmedabad,"[AV00099, AV00180, AV00095, AV00165, AV00153, ...",20
4,jaipur,"[QV00180, QV00023, QV00079, QV00152, QV00194, ...",17


In [11]:
agg_df

,foc_receiver_city,Delivered_By_Hub_Code,list_length
0,chennai,"[CV00145, CV00098, CV00153, CV00014, CV00101, CV00127, CV00115, CV00128, CV00147, CV00093, CV00103, CV00136, CV00151, CV00130, CV00102, CV00149, CV00054, CV00005, CV00111, CV00150, CV00154, CV00096, CV00112, CV00114, CV00109, CV00072, CV00004, CV00131, CV00107, CV00104, CV00117]",31
1,mumbai,"[MV00277, MV00134, MV00125, MV00254, MV00249, MV00275, MV00220, MV00280, MV00235, MV00278, MV00140, CV00141, MV00230, MV00219, MV00160, MV00128, MV00251, MV00153, MV00069, MV00268, MV00004, MV00064, MV1887, MV00223, MV00279, MV00270, MV00226, MV00049, MV00218, MV00281]",30
2,kolkata,"[KV00188, KV00185, KV00013, KV00098, KV00081, KV00012, KV00015, KV00030, KV00088, KV00197, KV00017, KV00172, KV00023, KV00099, KV00186, KV00125, KV00107, KV00097, KV00011, KV00110, KV00171, KV00126, KV00100, KV00106, KV00101, KV00085]",26
3,ahmedabad,"[AV00099, AV00180, AV00095, AV00165, AV00153, AV00172, AV00122, AV00098, AV00166, AV00150, AV00149, AV00096, AV00171, AV00185, AV00156, AV00102, AV00169, AV00174, AV00173, AV00168]",20
4,jaipur,"[QV00180, QV00023, QV00079, QV00152, QV00194, QV00191, QV00155, QV00165, QV00161, QV00196, QV00184, QV00064, QV00183, QV00179, QV00182, QV00198, QV00126]",17
5,delhi,"[NV00166, NV00024, NV00199, SV00080, NV099, NV00025, NV00018, SV00098, NV00165, NV00137, NV00201, NV00014, SV00066, NV00136]",14
6,varanasi,"[UV00179, UV00185, UV00153, UV00202, UV00122, UV00034, UV00028, UV00015, UV00213, UV00029, UV00035, UV00200, UV00159]",13
7,lucknow,"[UV00014, UV00147, UV00091, UV00021, UV00182, UV00169, UV00198, UV00157, UV00208, UV00211, UV00150, UV00131]",12
8,patna,"[TV00050, TV00006, TV00058, TV00054, TV00013, TV00019, TV00014, TV00012, TV00004, TV00059]",10
9,pune,"[PV00055, PV00048, PV00049, PV00081, PV00054, CV00150, PV00061, PV00057, PV00051, PV00071]",10


# model result collate

In [3]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

def extract_accuracy_from_report(file_path):
    """Extract overall accuracy from accuracy report CSV files"""
    try:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            # Look for the row with 'accuracy' in the class column
            if 'class' in df.columns:
                accuracy_row = df[df['class'] == 'accuracy']
                if not accuracy_row.empty and 'precision' in df.columns:
                    # Get the accuracy value from the precision column (they're all the same for accuracy row)
                    accuracy_value = accuracy_row['precision'].iloc[0]
                    return round(accuracy_value, 4)
                else:
                    return None
            else:
                return None
        else:
            return None
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

def count_lines_in_csv(file_path):
    """Count the number of lines in a CSV file (excluding header)"""
    try:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            first_col = df.iloc[:, 0]
            unique_values = set(first_col)
            return len(unique_values)
        
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return 0

def collate_model_results():
    """Collate results from all model folders"""
    
    # Path to models directory
    models_dir = Path("pincode_models")
    
    if not models_dir.exists():
        print(f"Models directory not found: {models_dir}")
        return
    
    # List to store results
    results = []
    
    # Get all subdirectories in models folder
    model_folders = [d for d in models_dir.iterdir() if d.is_dir()]
    
    print(f"Found {len(model_folders)} model folders")
    
    for folder in model_folders:
        folder_name = folder.name
        print(f"Processing folder: {folder_name}")
        
        # Paths to required files
        val_acc_file = folder / "val_accuracy_report.csv"
        test_acc_file = folder / "test_acurracy_report.csv"
        trained_cp_file = folder / "trained_cp_path.csv"
        other_cp_file = folder / "other_cp_path.csv"
        
        # Extract validation accuracy
        val_accuracy = extract_accuracy_from_report(val_acc_file)
        
        # Extract test accuracy
        test_accuracy = extract_accuracy_from_report(test_acc_file)
        
        # Count trained CP path classes
        trained_cp_count = count_lines_in_csv(trained_cp_file)
        
        # Count other CP path classes
        other_cp_count = count_lines_in_csv(other_cp_file)
        
        # Store results
        results.append({
            'Folder_Name': folder_name,
            'Validation_Accuracy': val_accuracy,
            'Test_Accuracy': test_accuracy,
            'Trained_CP_Path_Classes': trained_cp_count,
            'Other_CP_Path_Classes': other_cp_count
        })
    
    # Create DataFrame
    results_df = pd.DataFrame(results)
    
    # Sort by folder name (numerically)
    results_df['Folder_Name_Numeric'] = pd.to_numeric(results_df['Folder_Name'], errors='coerce')
    results_df = results_df.sort_values('Folder_Name_Numeric').drop('Folder_Name_Numeric', axis=1)
    
    # Save to Excel
    output_file = "model_results_collated.xlsx"
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Save main results summary
        results_df.to_excel(writer, sheet_name='Model_Results', index=False)
        
        # Save only test accuracy data from each folder to separate sheets
        for folder in model_folders:
            folder_name = folder.name
            print(f"Creating test accuracy sheet for folder: {folder_name}")
            
            # Path to test accuracy file
            test_acc_file = folder / "test_acurracy_report.csv"
            
            # Read and save test accuracy report only
            if os.path.exists(test_acc_file):
                try:
                    test_df = pd.read_csv(test_acc_file)
                    test_df.to_excel(writer, sheet_name=f'{folder_name}_test_accuracy', index=False)
                except Exception as e:
                    print(f"Error reading test accuracy for {folder_name}: {e}")
        
        # Add summary statistics
        summary_data = {
            'Metric': ['Total Models', 'Avg Validation Accuracy', 'Avg Test Accuracy', 
                      'Total Trained CP Classes', 'Total Other CP Classes'],
            'Value': [
                len(results_df),
                round(results_df['Validation_Accuracy'].mean(), 4) if results_df['Validation_Accuracy'].notna().any() else 'N/A',
                round(results_df['Test_Accuracy'].mean(), 4) if results_df['Test_Accuracy'].notna().any() else 'N/A',
                results_df['Trained_CP_Path_Classes'].sum(),
                results_df['Other_CP_Path_Classes'].sum()
            ]
        }
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    print(f"\nResults saved to: {output_file}")
    print(f"Total models processed: {len(results_df)}")
    
    # Display summary
    print("\nSummary of results:")
    print(results_df.describe())
    
    return results_df

# if __name__ == "__main__":
    # Change to the correct directory
script_dir = '/Users/amanbatra/Documents/pan_India_data/models'
os.chdir(script_dir)
    
    # Run the collation
results = collate_model_results()
    

Found 29 model folders
Processing folder: 57
Processing folder: 56
Processing folder: 51
Processing folder: 58
Processing folder: 20
Processing folder: 18
Processing folder: 27
Processing folder: 11
Processing folder: 16
Processing folder: 45
Processing folder: 17
Processing folder: 19
Processing folder: 26
Processing folder: 21
Processing folder: 44
Processing folder: 53
Processing folder: 52
Processing folder: 46
Processing folder: 48
Processing folder: 24
Processing folder: 23
Processing folder: 15
Processing folder: 12
Processing folder: 49
Processing folder: 47
Processing folder: 13
Processing folder: 14
Processing folder: 22
Processing folder: 25
Creating test accuracy sheet for folder: 57
Creating test accuracy sheet for folder: 56
Creating test accuracy sheet for folder: 51
Creating test accuracy sheet for folder: 58
Creating test accuracy sheet for folder: 20
Creating test accuracy sheet for folder: 18
Creating test accuracy sheet for folder: 27
Creating test accuracy sheet fo

In [5]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

def extract_accuracy_from_report(file_path):
    """Extract overall accuracy from accuracy report CSV files"""
    try:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            # Look for the row with 'accuracy' in the class column
            if 'class' in df.columns:
                accuracy_row = df[df['class'] == 'accuracy']
                if not accuracy_row.empty and 'precision' in df.columns:
                    # Get the accuracy value from the precision column (they're all the same for accuracy row)
                    accuracy_value = accuracy_row['precision'].iloc[0]
                    return round(accuracy_value, 4)
                else:
                    return None
            else:
                return None
        else:
            return None
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

def count_lines_in_csv(file_path):
    """Count the number of lines in a CSV file (excluding header)"""
    try:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            first_col = df.iloc[:, 0]
            unique_values = set(first_col)
            return len(unique_values)
        
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return 0

def collate_model_results():
    """Collate results from all model folders and add other_cp_path.csv content to a new sheet"""
    
    # Path to models directory
    models_dir = Path("pincode_models")
    
    if not models_dir.exists():
        print(f"Models directory not found: {models_dir}")
        return
    
    # List to store results
    results = []
    
    # List to store other_cp_path.csv content for all folders
    other_cp_path_rows = []
    
    # Get all subdirectories in models folder
    model_folders = [d for d in models_dir.iterdir() if d.is_dir()]
    
    print(f"Found {len(model_folders)} model folders")
    
    for folder in model_folders:
        folder_name = folder.name
        print(f"Processing folder: {folder_name}")
        
        # Paths to required files
        val_acc_file = folder / "val_accuracy_report.csv"
        test_acc_file = folder / "test_acurracy_report.csv"
        trained_cp_file = folder / "trained_cp_path.csv"
        other_cp_file = folder / "other_cp_path.csv"
        
        # Extract validation accuracy
        val_accuracy = extract_accuracy_from_report(val_acc_file)
        
        # Extract test accuracy
        test_accuracy = extract_accuracy_from_report(test_acc_file)
        
        # Count trained CP path classes
        trained_cp_count = count_lines_in_csv(trained_cp_file)
        
        # Count other CP path classes
        other_cp_count = count_lines_in_csv(other_cp_file)
        
        # Store results
        results.append({
            'Folder_Name': folder_name,
            'Validation_Accuracy': val_accuracy,
            'Test_Accuracy': test_accuracy,
            'Trained_CP_Path_Classes': trained_cp_count,
            'Other_CP_Path_Classes': other_cp_count
        })
        
        # Read other_cp_path.csv and append its values with folder name
        if os.path.exists(other_cp_file):
            try:
                df_other = pd.read_csv(other_cp_file)
                # For each row in the csv, add a row with folder name and the row as a dict
                for idx, row in df_other.iterrows():
                    # If the csv has only one column, just add its value
                    if len(df_other.columns) == 1:
                        other_cp_path_rows.append({
                            'Folder_Name': folder_name,
                            'Value': row.iloc[0]
                        })
                    else:
                        # If multiple columns, store as dict or tuple
                        other_cp_path_rows.append({
                            'Folder_Name': folder_name,
                            'Value': tuple(row.values)
                        })
            except Exception as e:
                print(f"Error reading other_cp_path.csv for {folder_name}: {e}")
        else:
            # If file does not exist, you may want to log or skip
            pass
    
    # Create DataFrame
    results_df = pd.DataFrame(results)
    
    # Sort by folder name (numerically)
    results_df['Folder_Name_Numeric'] = pd.to_numeric(results_df['Folder_Name'], errors='coerce')
    results_df = results_df.sort_values('Folder_Name_Numeric').drop('Folder_Name_Numeric', axis=1)
    
    # Create DataFrame for other_cp_path.csv content
    other_cp_path_df = pd.DataFrame(other_cp_path_rows)
    
    # Save to Excel
    output_file = "model_results_collated.xlsx"
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Save main results summary
        results_df.to_excel(writer, sheet_name='Model_Results', index=False)
        
        # Save only test accuracy data from each folder to separate sheets
        for folder in model_folders:
            folder_name = folder.name
            print(f"Creating test accuracy sheet for folder: {folder_name}")
            
            # Path to test accuracy file
            test_acc_file = folder / "test_acurracy_report.csv"
            
            # Read and save test accuracy report only
            if os.path.exists(test_acc_file):
                try:
                    test_df = pd.read_csv(test_acc_file)
                    test_df.to_excel(writer, sheet_name=f'{folder_name}_test_accuracy', index=False)
                except Exception as e:
                    print(f"Error reading test accuracy for {folder_name}: {e}")
        
        # Add summary statistics
        summary_data = {
            'Metric': ['Total Models', 'Avg Validation Accuracy', 'Avg Test Accuracy', 
                      'Total Trained CP Classes', 'Total Other CP Classes'],
            'Value': [
                len(results_df),
                round(results_df['Validation_Accuracy'].mean(), 4) if results_df['Validation_Accuracy'].notna().any() else 'N/A',
                round(results_df['Test_Accuracy'].mean(), 4) if results_df['Test_Accuracy'].notna().any() else 'N/A',
                results_df['Trained_CP_Path_Classes'].sum(),
                results_df['Other_CP_Path_Classes'].sum()
            ]
        }
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        
        # Add the other_cp_path.csv content to a new sheet
        # If the Value column contains tuples (multiple columns), expand them
        if not other_cp_path_df.empty:
            if other_cp_path_df['Value'].apply(lambda x: isinstance(x, tuple)).any():
                # Find the max tuple length
                max_len = other_cp_path_df['Value'].apply(lambda x: len(x) if isinstance(x, tuple) else 1).max()
                # Expand tuples into separate columns
                expanded = pd.DataFrame(other_cp_path_df['Value'].tolist(), columns=[f'Col_{i+1}' for i in range(max_len)])
                expanded.insert(0, 'Folder_Name', other_cp_path_df['Folder_Name'])
                expanded.to_excel(writer, sheet_name='Other_CP_Path', index=False)
            else:
                # Just two columns: Folder_Name and Value
                other_cp_path_df.to_excel(writer, sheet_name='Other_CP_Path', index=False)
        else:
            # If no data, write an empty sheet
            pd.DataFrame(columns=['Folder_Name', 'Value']).to_excel(writer, sheet_name='Other_CP_Path', index=False)
    
    print(f"\nResults saved to: {output_file}")
    print(f"Total models processed: {len(results_df)}")
    
    # Display summary
    print("\nSummary of results:")
    print(results_df.describe())
    
    return results_df

# if __name__ == "__main__":
    # Change to the correct directory
script_dir = '/Users/amanbatra/Documents/pan_India_data/models'
os.chdir(script_dir)
    
    # Run the collation
results = collate_model_results()

Found 29 model folders
Processing folder: 57
Processing folder: 56
Processing folder: 51
Processing folder: 58
Processing folder: 20
Processing folder: 18
Processing folder: 27
Processing folder: 11
Processing folder: 16
Processing folder: 45
Processing folder: 17
Processing folder: 19
Processing folder: 26
Processing folder: 21
Processing folder: 44
Processing folder: 53
Processing folder: 52
Processing folder: 46
Processing folder: 48
Processing folder: 24
Processing folder: 23
Processing folder: 15
Processing folder: 12
Processing folder: 49
Processing folder: 47
Processing folder: 13
Processing folder: 14
Processing folder: 22
Processing folder: 25
Creating test accuracy sheet for folder: 57
Creating test accuracy sheet for folder: 56
Creating test accuracy sheet for folder: 51
Creating test accuracy sheet for folder: 58
Creating test accuracy sheet for folder: 20
Creating test accuracy sheet for folder: 18
Creating test accuracy sheet for folder: 27
Creating test accuracy sheet fo